<a href="https://colab.research.google.com/github/nateraw/stable-diffusion-videos/blob/package/stable_diffusion_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion Videos

This notebook allows you to generate videos by interpolating the latent space of [Stable Diffusion](https://github.com/CompVis/stable-diffusion).

You can either dream up different versions of the same prompt, or morph between different text prompts (with seeds set for each for reproducibility).

If you like this notebook:
- consider giving the [repo a star](https://github.com/nateraw/stable-diffusion-videos) ⭐️
- consider following me on Github [@nateraw](https://github.com/nateraw) 

You can file any issues/feature requests [here](https://github.com/nateraw/stable-diffusion-videos/issues)

Enjoy 🤗

## Setup

In [4]:
! pip install stable_diffusion_videos
! git config --global credential.helper store

## Authenticate with Hugging Face Hub

You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work. For more information on access tokens, please refer to [this section of the documentation](https://huggingface.co/docs/hub/security-tokens).

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Run the App 🚀

### Load the Interface

This step will take a couple minutes the first time you run it.

In [ ]:
from stable_diffusion_videos import interface

### Launch

This cell launches a Gradio Interface. Here's how I suggest you use it:

1. Use the "Images" tab to generate images you like.
    - Find two images you want to morph between
    - These images should use the same settings (guidance scale, height, width)
    - Keep track of the seeds/settings you used so you can reproduce them

2. Generate videos using the "Videos" tab
    - Using the images you found from the step above, provide the prompts/seeds you recorded
    - Set the `num_walk_steps` - for testing you can use a small number like 3 or 5, but to get great results you'll want to use something larger (60-200 steps). 
    - You can (and should) use the `name` input to separate out where the images/videos are saved. 

💡 **Pro tip** - Click the link that looks like `https://<id-number>.gradio.app` below , and you'll be able to view it in full screen.

In [13]:
interface.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://16071.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7f030688f350>,
 'http://127.0.0.1:7860/',
 'https://16071.gradio.app')

---

## Use `walk` programatically

The other option is to not use the interface, and instead use `walk` programatically. Here's how you would do that...

First we define a helper fn for visualizing videos in colab

In [14]:
from IPython.display import HTML
from base64 import b64encode

def visualize_video_colab(video_path):
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

Walk! 🚶‍♀️

In [6]:
from stable_diffusion_videos import walk

video_path = walk(['a cat', 'a dog'], [42, 1337], num_steps=3, make_video=True)
visualize_video_colab(video_path)

COUNT: 1/3


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]